In [2]:
#opencv python 부분
import cv2   #opencv import

cap = cv2.VideoCapture(0)  #parameta 0은 컴퓨터와 연결된 usb cam설정을 불러옴

ret, frame = cap.read()    #Style transfer 의 입력값의 크기를 설정하기 위해 한프레임을 가져온다.
print(frame.shape)         #usb cam 이미지 크기 출력

#opencv python 부분


from __future__ import print_function
import sys
sys.path.insert(0, 'src')

import transform, numpy as np, vgg, pdb, os
import scipy.misc
import tensorflow as tf
import numpy

batch_size = 1


g = tf.Graph()    #tensor 그래프 초기화
soft_config = tf.ConfigProto(allow_soft_placement=True)
#명시된 디바이스가 존재하지 않는 경우 실행 디바이스를 TensorFlow가 자동으로 존재하는 디바이 스 중 선택
soft_config.gpu_options.allow_growth = True  
# session에서 필요로하는 메모리를 최소한으로 할당한 후 필요할 때마다 점진적으로 늘려가면서 메모리를 추가할당

#checkpoint_dir = "checkpoints"
#checkpoint_dir = "checkpoints/la_muse.ckpt"
#checkpoint_dir = "checkpoints/rain_princess.ckpt"
#checkpoint_dir = "checkpoints/scream.ckpt"
#checkpoint_dir = "checkpoints/udnie.ckpt"
#checkpoint_dir = "checkpoints/wave.ckpt"
checkpoint_dir = "checkpoints/wreck.ckpt"

#g.device('/gpu:0') => 학습할때는 GPU를 쓰는게 좋지만 일반적인 트레이닝 데이터로 결과를 만들어 낼때는 CPU를 사용하는것이 더 빠르다
with g.as_default(), g.device('/cpu:0'), tf.Session(config=soft_config) as sess:
    batch_shape = (batch_size,) +  frame.shape
    #batch_shape = (1, height, width , channel)
    
    img_placeholder = tf.placeholder(tf.float32, shape=batch_shape, name='img_placeholder')

    preds = transform.net(img_placeholder)
    
    #train 된 style 효과들을 불러온다
    saver = tf.train.Saver()
    if os.path.isdir(checkpoint_dir):
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            saver.restore(sess, ckpt.model_checkpoint_path)
        else:
            raise Exception("No checkpoint found...")
    else:
        saver.restore(sess, checkpoint_dir)
    #train 된 style 효과들을 불러온다
    
    X = np.zeros(batch_shape, dtype=np.float32)
    #input 데이터 크기를 할당한다
    
    while(True):
        #cap에서 이미지를 불러온다
        #ret는 이미지를 불러오는게 성공할 경우 1, 아니면 0
        #frame 는 이미지
        ret, frame = cap.read()
        
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        #X는 (1, height, width , channel) 이기 때문에 X[0]으로 집어 넣어야 한다. (X[0] = ( height, width , channel) )
        X[0] = frame
        
        _preds = sess.run(preds, feed_dict={img_placeholder:X})        
        #_preds.shape = (1, height, width , channel) 
        
        #np.clip 는 현재 설덩 해놓은 0 과  255사이의 값으로 바꿔준다.
        image = np.clip(_preds[0], 0, 255).astype(np.uint8)
        
        
        # Display the resulting frame
        cv2.imshow('frame',image)

        if cv2.waitKey(30) & 0xFF == ord('q'): #waitKey의 value의 값이 너무 높으면 너무 자주 key값 확인을 해서 느려짐, 프레임에 맞게 설정
            break



cap.release()
cv2.destroyAllWindows()

(480, 640, 3)
